In the previous notebook, we retrieved VOEvents related to a *Swift* XRT detection using the cone-search filter. A cone-search can bring to light new associations between VOEvents from different observatories, but we saw that it can also return unrelated events that just happen to lie nearby.

**
This notebook shows how to explore relations between VOEvents which are *already encoded by the packet author*, using the 'citation' mechanism.
**

It also includes a short example using `networkx` to visualise a citation-network.


In [ ]:
from __future__ import print_function

In [ ]:
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1
from voeventdb.remote.apiv1 import FilterKeys
from voeventdb.remote.helpers import Synopsis

Once again, we'll retrieve a handy example VOEvent representing a *Swift* XRT detection of a GRB:

In [ ]:
xrt_synopsis = Synopsis(apiv1.packet_synopsis('ivo://nasa.gsfc.gcn/SWIFT#XRT_Pos_666352-553'))

## References and citations

### Note on terminology:
In ``voeventdb``, we use the terms 'reference' and 'citation' in their precise bibliographic sense, i.e.
**references are made** (*to* another packet), and **citations are received** (*from* other packets).

### References made by a packet
It's easy to inspect the references made by a packet - it's right there in the synopsis:

In [ ]:
print("This packet contains {} references.".format(len(xrt_synopsis.references)))
xrt_synopsis.references[0]

### Citations received 
We can also check if this packet receives any citations. To do so, we search for *other* VOEvents which list it as a reference:

In [ ]:
my_filters = {FilterKeys.ref_exact:xrt_synopsis.ivorn}
apiv1.list_ivorn(my_filters)

Nope! No citations. But what about the original BAT trigger, the one referenced by the XRT VOEvent?

In [ ]:
bat_trigger_ivorn = xrt_synopsis.references[0]['ref_ivorn']
my_filters = {FilterKeys.ref_exact:bat_trigger_ivorn}
citations_to_bat = apiv1.list_ivorn(my_filters)
citations_to_bat

### Number of citations received, batch version:
Aha! So, the original BAT GRB trigger is the 'anchor reference' for all these other packets. Are any of them cited, in turn? There's a quick way to find out; we can use the ``ivorn_cited_count`` endpoint to get a citation count for all the packets matching our current filter set:

In [ ]:
my_filters

In [ ]:
apiv1.list_ivorn_ncites(my_filters)

Again, a big fat nope - they all have zero citations. 

### Party trick - network mapping
So we have quite a boring citation network - several packets cite the BAT GRB position, then the trail ends. Nonetheless, we can use it to show off a party trick - ``voeventdb.remote`` contains an extra function that makes repeated calls to the server to perform a 
[depth-first search](https://en.wikipedia.org/wiki/Depth-first_search) (with configurable maximum recursion level) of the citation network:

In [ ]:
from voeventdb.remote.apiv1.convenience import citation_network_map
cite_map = citation_network_map(xrt_synopsis.ivorn)

In [ ]:
cite_map

If we use [matplotlib](http://matplotlib.org/) and [networkx](https://networkx.github.io/), we can even draw the citation map. You can see the 'BAT_GRB' packet with many edges, representing citations, leading out from it

*(To do: More elaborate plots, with readable labels / colouring, etc. Contributions welcome!)*

In [ ]:
%matplotlib inline
import networkx as nx
G=nx.DiGraph()
for ivorn in cite_map.keys():
    G.add_node(ivorn)
for origin_ivorn, citing_ivorns in cite_map.items():
    for ci in citing_ivorns:
        G.add_edge(ivorn,ci)
nx.draw_networkx(G, arrows=True, with_labels=False)